## Step 1 – Data load, join keys, consistency & variable classification

This notebook:
- Loads and inspects the three datasets
- Identifies join keys between files
- Verifies session and user consistency
- Classifies variables as **Subjective** / **Behavioral** / **Experimental**

Outputs: **clean dataset summary** and **variable description table**.

### 1. Load and inspect datasets

In [ ]:
import pandas as pd
from pathlib import Path

BASE = Path("..")  # parent folder: Understanding User Behavior

logs = pd.read_csv(BASE / "logs.csv")
sessions = pd.read_csv(BASE / "sessions.csv")
questionnaires = pd.read_csv(BASE / "questionnaires.csv")

In [ ]:
# Shape and columns
print("=== logs ===")
print(logs.shape)
print(logs.columns.tolist())
print("\n=== sessions ===")
print(sessions.shape)
print(sessions.columns.tolist())
print("\n=== questionnaires ===")
print(questionnaires.shape)
print(questionnaires.columns.tolist())

In [ ]:
# Sample rows
display(logs.head(3))
display(sessions.head(3))
display(questionnaires.head(3))

In [ ]:
# Missing values
missing = pd.DataFrame({
    "file": ["logs", "sessions", "questionnaires"],
    "missing_count": [
        logs.isna().sum().sum(),
        sessions.isna().sum().sum(),
        questionnaires.isna().sum().sum(),
    ],
})
missing

### 2. Identify join keys

- **logs ↔ sessions:** `session_id` (each event belongs to one session; each session has one row in sessions)
- **sessions ↔ questionnaires:** `user_id` (each session is associated with one user; each user has a row in questionnaires)

In [ ]:
join_keys = pd.DataFrame([
    {"From": "logs", "To": "sessions", "Key": "session_id", "Description": "Event → session"},
    {"From": "sessions", "To": "questionnaires", "Key": "user_id", "Description": "Session → user / questionnaire"},
])
join_keys

In [ ]:
# Verify: every log session_id exists in sessions, and vice versa
log_sids = set(logs["session_id"])
sess_sids = set(sessions["session_id"])
only_in_logs = log_sids - sess_sids
only_in_sessions = sess_sids - log_sids

sess_uids = set(sessions["user_id"])
quest_uids = set(questionnaires["user_id"])
only_in_sessions_uid = sess_uids - quest_uids
only_in_quest = quest_uids - sess_uids

print("Join key coverage:")
print(f"  session_id: sessions in logs only = {len(only_in_logs)}, in sessions only = {len(only_in_sessions)}")
print(f"  user_id: users in sessions only = {len(only_in_sessions_uid)}, in questionnaires only = {len(only_in_quest)}")

### 3. Verify session consistency

In [ ]:
n_unique_log_sessions = logs["session_id"].nunique()
n_unique_sess_sessions = sessions["session_id"].nunique()
n_unique_sess_users = sessions["user_id"].nunique()
n_unique_quest_users = questionnaires["user_id"].nunique()

consistency = pd.DataFrame([
    {"Check": "Unique session_id in logs", "Value": n_unique_log_sessions},
    {"Check": "Unique session_id in sessions", "Value": n_unique_sess_sessions},
    {"Check": "Sessions in logs but not in sessions", "Value": len(only_in_logs)},
    {"Check": "Sessions in sessions but not in logs", "Value": len(only_in_sessions)},
    {"Check": "Unique user_id in sessions", "Value": n_unique_sess_users},
    {"Check": "Unique user_id in questionnaires", "Value": n_unique_quest_users},
    {"Check": "Users in sessions but not in questionnaires", "Value": len(only_in_sessions_uid)},
    {"Check": "Users in questionnaires but not in sessions", "Value": len(only_in_quest)},
])
consistency

In [ ]:
# Sessions: one user_id and one interface_version per session_id?
per_session = sessions.groupby("session_id").agg({
    "user_id": "nunique",
    "interface_version": "nunique",
})
sessions_with_multiple = (per_session > 1).any(axis=1).sum()
print(f"Sessions with more than one user_id or interface_version: {sessions_with_multiple}")
print("Conclusion: datasets are consistent for joins on session_id and user_id." if sessions_with_multiple == 0 and len(only_in_logs) == 0 and len(only_in_sessions) == 0 else "Review needed.")

### 4. Classify variables: Subjective / Behavioral / Experimental

In [ ]:
# Variable description table (Source, Category, Description)
variable_descriptions = [
    # Behavioral (logs)
    ("log_id", "logs.csv", "Behavioral", "Unique identifier of each log event"),
    ("session_id", "logs.csv", "Behavioral", "Session identifier for the event (links to sessions.csv)"),
    ("timestamp", "logs.csv", "Behavioral", "Date and time when the event occurred"),
    ("action_type", "logs.csv", "Behavioral", "Type of user action (click, drill_down, filter, hover)"),
    ("element_id", "logs.csv", "Behavioral", "Interface element (chart_trend, KPI_costs, KPI_sales, table_details)"),
    ("duration_sec", "logs.csv", "Behavioral", "Duration of the action in seconds"),
    ("error_flag", "logs.csv", "Behavioral", "Whether an error occurred during the event (0/1)"),
    ("decision_made", "logs.csv", "Behavioral", "Whether a decision was made at this event (0/1)"),
    # Experimental (sessions)
    ("session_id", "sessions.csv", "Experimental", "Identifier of the session (links to logs.csv)"),
    ("user_id", "sessions.csv", "Experimental", "Identifier of the user (links to questionnaires.csv)"),
    ("interface_version", "sessions.csv", "Experimental", "Version of the dashboard shown (A or B)"),
    # Key + Subjective (questionnaires)
    ("user_id", "questionnaires.csv", "Key", "User identifier (join key with sessions.csv)"),
    ("SUS_score", "questionnaires.csv", "Subjective", "Perceived usability (System Usability Scale)"),
    ("NASA_TLX", "questionnaires.csv", "Subjective", "Self-reported cognitive load"),
    ("UES_engagement", "questionnaires.csv", "Subjective", "Self-reported engagement with the system"),
    ("IMI_autonomy", "questionnaires.csv", "Subjective", "Perceived autonomy using the system"),
    ("IMI_competence", "questionnaires.csv", "Subjective", "Perceived competence using the system"),
    ("intention_reuse", "questionnaires.csv", "Subjective", "Intention to reuse the dashboard in the future"),
]

var_table = pd.DataFrame(
    variable_descriptions,
    columns=["Variable", "Source file", "Category", "Description"]
)
var_table

### 5. Clean dataset summary

In [ ]:
dataset_summary = pd.DataFrame([
    {
        "Dataset": "logs",
        "Rows": len(logs),
        "Columns": len(logs.columns),
        "Missing values": logs.isna().sum().sum(),
        "Notes": "Event-level behavior",
    },
    {
        "Dataset": "sessions",
        "Rows": len(sessions),
        "Columns": len(sessions.columns),
        "Missing values": sessions.isna().sum().sum(),
        "Notes": "One row per session; experimental context",
    },
    {
        "Dataset": "questionnaires",
        "Rows": len(questionnaires),
        "Columns": len(questionnaires.columns),
        "Missing values": questionnaires.isna().sum().sum(),
        "Notes": "One row per user; subjective measures",
    },
])
dataset_summary

### 6. Variable description table (for reporting)

In [ ]:
# Same table as above, formatted for export/report
var_table